# Decoder Visualization and Analysis

This notebook demonstrates:
1. Loading the trained model and extracting just the decoder part
2. Loading the saved bottleneck tensor from inference results
3. Passing the bottleneck through the decoder
4. Visualizing the decoder outputs and analyzing the reconstruction process

## Overview
- **Input**: Bottleneck tensor from encoder (compressed representation)
- **Process**: Decoder-only inference (upsampling and reconstruction)
- **Output**: Reconstructed spectral bands with detailed visualizations

## 1. Setup and Imports

In [43]:
import os
import sys
import warnings
from pathlib import Path

# Data processing
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Project imports
sys.path.append('../')
from model_zoo.models import define_model
from utils.utils import load_config

# Configure display
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA device: NVIDIA L40S


## 2. Configuration and Paths

In [44]:
# Configuration paths
result_path = "/home/ubuntu/project/sentinel-2-ai-compressor/src/results/2025-08-19_22-18-59"
checkpoint_path = f"{result_path}/checkpoints"
config_path = f"{result_path}/config.yaml"

# Data paths
inference_results_dir = "/home/ubuntu/project/sentinel-2-ai-compressor/src/inference_results"
bottleneck_path = f"/home/ubuntu/project/sentinel-2-ai-compressor/src/inference_results/_bottleneck.pt"

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Verify paths exist
print(f"\nPath verification:")
print(f"Config exists: {os.path.exists(config_path)}")
print(f"Checkpoint dir exists: {os.path.exists(checkpoint_path)}")
print(f"Bottleneck file exists: {os.path.exists(bottleneck_path)}")

Using device: cuda

Path verification:
Config exists: True
Checkpoint dir exists: True
Bottleneck file exists: True


## 3. Load Model Configuration and Extract Decoder

In [45]:
# Load configuration
config = load_config(config_path)

print("Model Configuration:")
print(f"Model: {config['MODEL']['model_name']}")
print(f"Encoder: {config['MODEL']['encoder_name']}")
print(f"Input channels: {len(config['DATASET']['bands'])}")
print(f"Output channels: {len(config['DATASET']['bands'])}")
print(f"Activation: {config['MODEL']['activation']}")
print(f"Bands: {config['DATASET']['bands']}")

bands = config['DATASET']['bands']

Model Configuration:
Model: Unet
Encoder: timm-efficientnet-b2
Input channels: 11
Output channels: 11
Activation: relu
Bands: ['b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b09', 'b11', 'b12', 'b8a']


In [46]:
# Initialize full model
full_model = define_model(
    name=config['MODEL']['model_name'],
    encoder_name=config['MODEL']['encoder_name'],
    encoder_weights=config['MODEL']['encoder_weights'],
    in_channel=len(config['DATASET']['bands']),
    out_channels=len(config['DATASET']['bands']),
    activation=config['MODEL']['activation']
)

# Load trained weights
print("Loading trained model weights...")
full_model.load_state_dict(torch.load(os.path.join(checkpoint_path, "best_model.pth")))
full_model = full_model.to(device)
full_model.eval()

print(f"Full model loaded successfully!")
print(f"Model architecture: {type(full_model).__name__}")

Loading trained model weights...
Full model loaded successfully!
Model architecture: Sequential


## 4. Load Bottleneck Tensor

In [51]:
# Load the bottleneck tensor
print("Loading bottleneck tensor...")
bottleneck_tensor = torch.load(bottleneck_path, map_location=device)

print(f"Bottleneck tensor loaded successfully!")
print(f"Bottleneck shape: {bottleneck_tensor.shape}")
print(f"Bottleneck dtype: {bottleneck_tensor.dtype}")
print(f"Bottleneck device: {bottleneck_tensor.device}")
print(f"Memory size: {bottleneck_tensor.numel() * bottleneck_tensor.element_size() / (1024**2):.2f} MB")

# Basic statistics
print(f"\nBottleneck Statistics:")
print(f"Mean: {bottleneck_tensor.mean().item():.6f}")
print(f"Std: {bottleneck_tensor.std().item():.6f}")
print(f"Min: {bottleneck_tensor.min().item():.6f}")
print(f"Max: {bottleneck_tensor.max().item():.6f}")

Loading bottleneck tensor...
Bottleneck tensor loaded successfully!
Bottleneck shape: torch.Size([36, 208, 10, 10])
Bottleneck dtype: torch.float32
Bottleneck device: cuda:0
Memory size: 2.86 MB

Bottleneck Statistics:
Mean: -0.128684
Std: 6.212026
Min: -115.062149
Max: 122.589828
